やっていること  
・HOMESスクレイピング（港区）  
・SUUMOスクレイピング（港区、渋谷区、品川区、大田区）  
・表記ゆれ調整  
・HOMESとSUUMOのマージ  
・重複削除  
・Google Geocoding APIで「名称」から緯度・経度算出  
・SQLite3にてDB構築

HOMESで「港区」物件をスクレイピング

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

base_url = "https://www.homes.co.jp/chintai/tokyo/minato-city/list/?page={}"
max_page = 5
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

all_data = []

for page in range(1, max_page + 1):
    url = base_url.format(page)
    response = requests.get(url, headers=headers) 
    soup = BeautifulSoup(response.content, 'lxml')
    items = soup.findAll("div", {"class": "mod-mergeBuilding--rent--photo"})

    print("page", page, "items", len(items))

    for item in items:
        base_data = {}
        base_data["名称"] = item.find(class_="bukkenName").get_text(strip=True) if item.find(class_="bukkenName") else None
        base_data["アドレス"] = next((td.find_next_sibling('td').get_text(strip=True) for td in item.find_all('th') if td.get_text(strip=True) == '所在地'), None)
        base_data["アクセス"] = item.find("td", class_="traffic").get_text(strip=True) if item.find("td", class_="traffic") else ', '.join(span.get_text(strip=True) for span in item.find_all("span", class_="prg-stationText"))
        module_body = item.find('div', class_='moduleBody')
        if module_body:
            construction_th = module_body.find('th', string='築年数/階数')
            if construction_th:
                base_data["築年数"] = construction_th.find_next_sibling('td').get_text(strip=True).split(' ')[0]
                base_data["構造"] = construction_th.find_next_sibling('td').get_text(strip=True).split(' ')[2]

        rooms = item.find_all(class_="unitListBody prg-unitListBody")
        for room in rooms:
            data = base_data.copy()
            
            # 当該部屋の階数
            floor_number_td = room.find(class_="roomKaisuu")
            data["階数"] = floor_number_td.get_text(strip=True) if floor_number_td else None

            # 賃料と管理費を分けて取得
            rent_price_label = room.select_one("span.priceLabel")
            rent_price = rent_price_label.get_text(strip=True) if rent_price_label else None
            rent_admin = rent_price_label.next_sibling.strip().replace("/", "").replace(",", "") if rent_price_label else None
            data["家賃"] = rent_price
            data["管理費"] = rent_admin

            # 敷金と礼金を分けて取得
            price = room.select_one("td.price")
            br_tag = price.find('br').next_sibling.strip()
            depo = br_tag.split("/")[0]
            key = br_tag.split("/")[1]
            data["敷金"] = depo
            data["礼金"] = key

            # 間取りと占有面積を分けて取得
            layout = room.select_one("td.layout")
            room_type = layout.contents[0].strip() if layout.contents else None
            room_area = layout.find('br').next_sibling.strip().replace('m²', 'm2') if layout.find('br') else None
            data["間取り"] = room_type
            data["面積"] = room_area

            property_image_element = item.select_one(".bukkenPhoto .photo img")
            data["物件画像URL"] = property_image_element["data-original"] if property_image_element else None

            # 間取り画像URL
            floor_plan_image_element = item.select_one(".floarPlanPic img")
            data["間取画像URL"] = floor_plan_image_element["data-original"] if floor_plan_image_element else None

            # 物件詳細URL
            property_link_element = item.select_one("a[href*='/chintai/room']")
            data["物件詳細URL"] = property_link_element['href'] if property_link_element else None


            all_data.append(data)

page 1 items 30
page 2 items 30
page 3 items 30
page 4 items 30
page 5 items 30


In [2]:
df = pd.DataFrame(all_data)
df.head(5)

,名称,アドレス,アクセス,築年数,構造,階数,家賃,管理費,敷金,礼金,間取り,面積,物件画像URL,間取画像URL,物件詳細URL
0,プラウドフラット虎ノ門,東京都港区西新橋3丁目12-9,"東京メトロ日比谷線 虎ノ門ヒルズ駅 徒歩7分, 都営三田線 御成門駅 徒歩7分",1年,15階建,2階,13.4万円,10000円,1ヶ月,無,ワンルーム,22.01m2,https://image1.homes.jp/smallimg/image.php?fil...,https://image1.homes.jp/smallimg/image.php?fil...,https://www.homes.co.jp/chintai/room/994725341...
1,パークアクシス芝浦キャナル,東京都港区海岸3丁目2-3,"新交通ゆりかもめ 芝浦ふ頭駅 徒歩7分, 新交通ゆりかもめ 日の出駅 徒歩9分, JR山手線...",3年,10階建,10階,28.2万円,20000円,1ヶ月,無,2LDK,56.01m2,https://image.homes.jp/smallimg/image.php?file...,https://image4.homes.jp/smallimg/image.php?fil...,https://www.homes.co.jp/chintai/room/2d64f3e82...
2,カナール芝浦,東京都港区芝浦2丁目,"JR山手線 田町駅 徒歩9分, 都営浅草線 三田駅 徒歩12分, ゆりかもめ 芝浦ふ頭 徒歩10分",2年,8階建,3階,13.2万円,10000円,1ヶ月,1ヶ月,ワンルーム,30.88m2,https://image3.homes.jp/smallimg/image.php?fil...,https://image.homes.jp/smallimg/image.php?file...,https://www.homes.co.jp/chintai/room/4d6b446a5...
3,JR山手線 田町駅 徒歩12分,東京都港区海岸3丁目2-3,"新交通ゆりかもめ 芝浦ふ頭駅 徒歩7分, 新交通ゆりかもめ 日の出駅 徒歩8分, JR山手線...",3年,10階建,10階,28.2万円,20000円,1ヶ月,無,2LDK,56.01m2,https://image3.homes.jp/smallimg/image.php?fil...,https://image4.homes.jp/smallimg/image.php?fil...,https://www.homes.co.jp/chintai/room/b308ff912...
4,京急本線 北品川駅 徒歩10分,東京都港区港南2丁目11-3,"京急本線 北品川駅 徒歩10分, 東京モノレール 天王洲アイル駅 徒歩12分, JR東海道本...",2年,15階建,8階,11.2万円,15000円,1ヶ月,無,ワンルーム,22.55m2,https://image4.homes.jp/smallimg/image.php?fil...,https://image4.homes.jp/smallimg/image.php?fil...,https://www.homes.co.jp/chintai/room/6be8adeeb...


In [3]:
len(df)

150

スーモと重複削除するために住所の表記を合わせる

In [4]:
# 住所の標準化関数
def standardize_address(address):
    # 数字を半角から全角に変換
    address = re.sub(r'1', '１', address)
    address = re.sub(r'2', '２', address)
    address = re.sub(r'3', '３', address)
    address = re.sub(r'4', '４', address)
    address = re.sub(r'5', '５', address)
    address = re.sub(r'6', '６', address)
    address = re.sub(r'7', '７', address)
    address = re.sub(r'8', '８', address)
    address = re.sub(r'9', '９', address)
    address = re.sub(r'0', '０', address)
    
    # 「丁目」以降を削除
    address = re.sub(r'丁目.*', '', address)
    address = re.sub(r'-.*', '', address)
    
    return address

# 標準化した住所を新しいカラムに追加
df['アドレス'] = df['アドレス'].apply(standardize_address)

#数字を除去した住所をアドレス_数字除去に格納
def remove_numbers(address):
    # 数字を削除
    address_no_numbers = re.sub(r'[0-9０-９]', '', address)
    return address_no_numbers

# 数字を削除した住所を新しいカラムに追加
df['アドレス_数字除去'] = df['アドレス'].apply(remove_numbers)

In [5]:
df.head(5)

,名称,アドレス,アクセス,築年数,構造,階数,家賃,管理費,敷金,礼金,間取り,面積,物件画像URL,間取画像URL,物件詳細URL,アドレス_数字除去
0,プラウドフラット虎ノ門,東京都港区西新橋３,"東京メトロ日比谷線 虎ノ門ヒルズ駅 徒歩7分, 都営三田線 御成門駅 徒歩7分",1年,15階建,2階,13.4万円,10000円,1ヶ月,無,ワンルーム,22.01m2,https://image1.homes.jp/smallimg/image.php?fil...,https://image1.homes.jp/smallimg/image.php?fil...,https://www.homes.co.jp/chintai/room/994725341...,東京都港区西新橋
1,パークアクシス芝浦キャナル,東京都港区海岸３,"新交通ゆりかもめ 芝浦ふ頭駅 徒歩7分, 新交通ゆりかもめ 日の出駅 徒歩9分, JR山手線...",3年,10階建,10階,28.2万円,20000円,1ヶ月,無,2LDK,56.01m2,https://image.homes.jp/smallimg/image.php?file...,https://image4.homes.jp/smallimg/image.php?fil...,https://www.homes.co.jp/chintai/room/2d64f3e82...,東京都港区海岸
2,カナール芝浦,東京都港区芝浦２,"JR山手線 田町駅 徒歩9分, 都営浅草線 三田駅 徒歩12分, ゆりかもめ 芝浦ふ頭 徒歩10分",2年,8階建,3階,13.2万円,10000円,1ヶ月,1ヶ月,ワンルーム,30.88m2,https://image3.homes.jp/smallimg/image.php?fil...,https://image.homes.jp/smallimg/image.php?file...,https://www.homes.co.jp/chintai/room/4d6b446a5...,東京都港区芝浦
3,JR山手線 田町駅 徒歩12分,東京都港区海岸３,"新交通ゆりかもめ 芝浦ふ頭駅 徒歩7分, 新交通ゆりかもめ 日の出駅 徒歩8分, JR山手線...",3年,10階建,10階,28.2万円,20000円,1ヶ月,無,2LDK,56.01m2,https://image3.homes.jp/smallimg/image.php?fil...,https://image4.homes.jp/smallimg/image.php?fil...,https://www.homes.co.jp/chintai/room/b308ff912...,東京都港区海岸
4,京急本線 北品川駅 徒歩10分,東京都港区港南２,"京急本線 北品川駅 徒歩10分, 東京モノレール 天王洲アイル駅 徒歩12分, JR東海道本...",2年,15階建,8階,11.2万円,15000円,1ヶ月,無,ワンルーム,22.55m2,https://image4.homes.jp/smallimg/image.php?fil...,https://image4.homes.jp/smallimg/image.php?fil...,https://www.homes.co.jp/chintai/room/6be8adeeb...,東京都港区港南


スーモからも出力　※出力は東京都港区、東京都渋谷区、東京都品川区、東京都大田区

In [6]:
# 基本URLと最大ページ数の設定
base_url = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&pc=30&smk=&po1=25&po2=99&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sc=13103&sc=13113&sc=13109&sc=13111&ta=13&cb=0.0&ct=9999999&et=9999999&mb=0&mt=9999999&cn=30&fw2="
max_page = 5  # 最大ページ数

all_data = []

for page in range(1, max_page + 1):
    url = base_url.format(page)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml')
    items = soup.findAll("div", {"class": "cassetteitem"})


    print("page", page, "items", len(items))

    for item in items:
        base_data = {}
        base_data["名称"]     = item.find("div", {"class": "cassetteitem_content-title"}).get_text(strip=True) if item.find("div", {"class": "cassetteitem_content-title"}) else None
        base_data["カテゴリ"] = item.find("div", {"class": "cassetteitem_content-label"}).span.get_text(strip=True) if item.find("div", {"class": "cassetteitem_content-label"}) else None
        base_data["アドレス"] = item.find("li", {"class": "cassetteitem_detail-col1"}).get_text(strip=True) if item.find("li", {"class": "cassetteitem_detail-col1"}) else None
        
        # 駅のアクセス情報をまとめて取得
        base_data["アクセス"] = ", ".join([station.get_text(strip=True) for station in item.findAll("div", {"class": "cassetteitem_detail-text"})])

        construction_info = item.find("li", {"class": "cassetteitem_detail-col3"}).find_all("div") if item.find("li", {"class": "cassetteitem_detail-col3"}) else None
        base_data["築年数"] = construction_info[0].get_text(strip=True) if construction_info and len(construction_info) > 0 else None
        base_data["構造"] = construction_info[1].get_text(strip=True) if construction_info and len(construction_info) > 1 else None

        tbodys = item.find("table", {"class": "cassetteitem_other"}).findAll("tbody")

        for tbody in tbodys:
            data = base_data.copy()
            # 階数情報の正確な取得
            floor_info = tbody.find_all("td")[2].get_text(strip=True) if len(tbody.find_all("td")) > 2 else None
            data["階数"]   = floor_info
            data["家賃"]   = tbody.select_one(".cassetteitem_price--rent").get_text(strip=True) if tbody.select_one(".cassetteitem_price--rent") else None
            data["管理費"] = tbody.select_one(".cassetteitem_price--administration").get_text(strip=True) if tbody.select_one(".cassetteitem_price--administration") else None
            data["敷金"]   = tbody.select_one(".cassetteitem_price--deposit").get_text(strip=True) if tbody.select_one(".cassetteitem_price--deposit") else None
            data["礼金"]   = tbody.select_one(".cassetteitem_price--gratuity").get_text(strip=True) if tbody.select_one(".cassetteitem_price--gratuity") else None
            data["間取り"] = tbody.select_one(".cassetteitem_madori").get_text(strip=True) if tbody.select_one(".cassetteitem_madori") else None
            data["面積"]   = tbody.select_one(".cassetteitem_menseki").get_text(strip=True) if tbody.select_one(".cassetteitem_menseki") else None

            # 物件画像・間取り画像・詳細URLの取得を最後に行う
            property_image_element = item.find(class_="cassetteitem_object-item")
            data["物件画像URL"] = property_image_element.img["rel"] if property_image_element and property_image_element.img else None

            floor_plan_image_element = tbody.find(class_="casssetteitem_other-thumbnail")
            data["間取画像URL"] = floor_plan_image_element.img["rel"] if floor_plan_image_element and floor_plan_image_element.img else None

            property_link_element = tbody.select_one("a[href*='/chintai/jnc_']")
            data["物件詳細URL"] = "https://suumo.jp" +property_link_element['href'] if property_link_element else None ## 不動産サイトから詳細URLリンクを読み解き作成

            all_data.append(data)    

page 1 items 30
page 2 items 30
page 3 items 30
page 4 items 30
page 5 items 30


スーモのデータはdf2へ

In [7]:
df2 = pd.DataFrame(all_data)

In [8]:
len(df2)

1216

In [9]:
df2.head(2)

,名称,カテゴリ,アドレス,アクセス,築年数,構造,階数,家賃,管理費,敷金,礼金,間取り,面積,物件画像URL,間取画像URL,物件詳細URL
0,ガーデン高輪,賃貸マンション,東京都港区高輪４,"都営浅草線/高輪台駅 歩8分, ＪＲ山手線/品川駅 歩10分, ＪＲ山手線/五反田駅 歩14分",築15年,地下2地上4階建,3階,168万円,-,336万円,168万円,5SLDK,324.17m2,https://img01.suumo.com/front/gazo/fr/bukken/5...,https://img01.suumo.com/front/gazo/fr/bukken/0...,https://suumo.jp/chintai/jnc_000068413792/?bc=...
1,ガーデン高輪,賃貸マンション,東京都港区高輪４,"都営浅草線/高輪台駅 歩8分, ＪＲ山手線/品川駅 歩10分, ＪＲ山手線/五反田駅 歩14分",築15年,地下2地上4階建,3階,200万円,-,400万円,200万円,4SLDK,324.17m2,https://img01.suumo.com/front/gazo/fr/bukken/5...,https://img01.suumo.com/front/gazo/fr/bukken/3...,https://suumo.jp/chintai/jnc_000091263459/?bc=...


In [10]:
#数字を除去した住所をアドレス_数字除去に格納
def remove_numbers(address):
    # 数字を削除
    address_no_numbers = re.sub(r'[0-9０-９]', '', address)
    return address_no_numbers

# 数字を削除した住所を新しいカラムに追加
df2['アドレス_数字除去'] = df2['アドレス'].apply(remove_numbers)

カテゴリはホームズにないので消す

In [11]:
df2_cleaned = df2.drop(columns=['カテゴリ'])

In [12]:
df2_cleaned.head(1)

,名称,アドレス,アクセス,築年数,構造,階数,家賃,管理費,敷金,礼金,間取り,面積,物件画像URL,間取画像URL,物件詳細URL,アドレス_数字除去
0,ガーデン高輪,東京都港区高輪４,"都営浅草線/高輪台駅 歩8分, ＪＲ山手線/品川駅 歩10分, ＪＲ山手線/五反田駅 歩14分",築15年,地下2地上4階建,3階,168万円,-,336万円,168万円,5SLDK,324.17m2,https://img01.suumo.com/front/gazo/fr/bukken/5...,https://img01.suumo.com/front/gazo/fr/bukken/0...,https://suumo.jp/chintai/jnc_000068413792/?bc=...,東京都港区高輪


dfとdf2_cleanを合体させる

In [13]:
df_merged = pd.concat([df, df2_cleaned], ignore_index=True)

In [14]:
df_merged.head(1)

,名称,アドレス,アクセス,築年数,構造,階数,家賃,管理費,敷金,礼金,間取り,面積,物件画像URL,間取画像URL,物件詳細URL,アドレス_数字除去
0,プラウドフラット虎ノ門,東京都港区西新橋３,"東京メトロ日比谷線 虎ノ門ヒルズ駅 徒歩7分, 都営三田線 御成門駅 徒歩7分",1年,15階建,2階,13.4万円,10000円,1ヶ月,無,ワンルーム,22.01m2,https://image1.homes.jp/smallimg/image.php?fil...,https://image1.homes.jp/smallimg/image.php?fil...,https://www.homes.co.jp/chintai/room/994725341...,東京都港区西新橋


In [15]:
len(df_merged)

1366

↑ホームズで抽出した物件数とスーモで抽出した物件数の合算の数字が合っていることを確認

重複物件を削除する。削除の条件は、「築年数」「構造」「階数」「家賃」「面積」「アドレス_数字除去」カラムが一致していること。

In [16]:
# 重複削除
df_deduplicated = df_merged.drop_duplicates(subset=['築年数', '構造', '階数', '家賃', '面積', 'アドレス_数字除去'])

# 削除前後の行数を比較
original_count = len(df_merged)
deduplicated_count = len(df_deduplicated)
duplicates_removed = original_count - deduplicated_count

# 結果の表示
print(f"\n削除された重複レコード数: {duplicates_removed}")


削除された重複レコード数: 1063


df_deduplicatedから「アドレス_数字除去」を削除する

In [17]:
df_deduplicated = df_deduplicated.drop(columns=['アドレス_数字除去'])
df_deduplicated.head(1)

,名称,アドレス,アクセス,築年数,構造,階数,家賃,管理費,敷金,礼金,間取り,面積,物件画像URL,間取画像URL,物件詳細URL
0,プラウドフラット虎ノ門,東京都港区西新橋３,"東京メトロ日比谷線 虎ノ門ヒルズ駅 徒歩7分, 都営三田線 御成門駅 徒歩7分",1年,15階建,2階,13.4万円,10000円,1ヶ月,無,ワンルーム,22.01m2,https://image1.homes.jp/smallimg/image.php?fil...,https://image1.homes.jp/smallimg/image.php?fil...,https://www.homes.co.jp/chintai/room/994725341...


「家賃」列から万円を削除してobject→floatに変換

In [18]:
def extract_number(x):
    try:
        return float(x.split('万円')[0].replace(',',''))
    except:
        return np.nan

In [19]:
df_deduplicated['家賃'] = df_deduplicated['家賃'].apply(extract_number).astype(float)

In [20]:
df_deduplicated.head(2)

,名称,アドレス,アクセス,築年数,構造,階数,家賃,管理費,敷金,礼金,間取り,面積,物件画像URL,間取画像URL,物件詳細URL
0,プラウドフラット虎ノ門,東京都港区西新橋３,"東京メトロ日比谷線 虎ノ門ヒルズ駅 徒歩7分, 都営三田線 御成門駅 徒歩7分",1年,15階建,2階,13.4,10000円,1ヶ月,無,ワンルーム,22.01m2,https://image1.homes.jp/smallimg/image.php?fil...,https://image1.homes.jp/smallimg/image.php?fil...,https://www.homes.co.jp/chintai/room/994725341...
1,パークアクシス芝浦キャナル,東京都港区海岸３,"新交通ゆりかもめ 芝浦ふ頭駅 徒歩7分, 新交通ゆりかもめ 日の出駅 徒歩9分, JR山手線...",3年,10階建,10階,28.2,20000円,1ヶ月,無,2LDK,56.01m2,https://image.homes.jp/smallimg/image.php?file...,https://image4.homes.jp/smallimg/image.php?fil...,https://www.homes.co.jp/chintai/room/2d64f3e82...


「区」列を追加する

In [21]:
# 東京都を削除して新しい列を作成
df_deduplicated['アドレス_修正'] = df_deduplicated['アドレス'].str.replace('東京都', '')

# 区の後の文字を削除して新しい列を作成
df_deduplicated['区'] = df_deduplicated['アドレス_修正'].str.split('区', expand=True)[0]+ '区'

In [22]:
df_deduplicated.head(2)

,名称,アドレス,アクセス,築年数,構造,階数,家賃,管理費,敷金,礼金,間取り,面積,物件画像URL,間取画像URL,物件詳細URL,アドレス_修正,区
0,プラウドフラット虎ノ門,東京都港区西新橋３,"東京メトロ日比谷線 虎ノ門ヒルズ駅 徒歩7分, 都営三田線 御成門駅 徒歩7分",1年,15階建,2階,13.4,10000円,1ヶ月,無,ワンルーム,22.01m2,https://image1.homes.jp/smallimg/image.php?fil...,https://image1.homes.jp/smallimg/image.php?fil...,https://www.homes.co.jp/chintai/room/994725341...,港区西新橋３,港区
1,パークアクシス芝浦キャナル,東京都港区海岸３,"新交通ゆりかもめ 芝浦ふ頭駅 徒歩7分, 新交通ゆりかもめ 日の出駅 徒歩9分, JR山手線...",3年,10階建,10階,28.2,20000円,1ヶ月,無,2LDK,56.01m2,https://image.homes.jp/smallimg/image.php?file...,https://image4.homes.jp/smallimg/image.php?fil...,https://www.homes.co.jp/chintai/room/2d64f3e82...,港区海岸３,港区


In [23]:
# df_deduplicatedから[アドレス_修正]を削除
df_deduplicated = df_deduplicated.drop(columns=['アドレス_修正'])

In [24]:
df_deduplicated.head(1)

,名称,アドレス,アクセス,築年数,構造,階数,家賃,管理費,敷金,礼金,間取り,面積,物件画像URL,間取画像URL,物件詳細URL,区
0,プラウドフラット虎ノ門,東京都港区西新橋３,"東京メトロ日比谷線 虎ノ門ヒルズ駅 徒歩7分, 都営三田線 御成門駅 徒歩7分",1年,15階建,2階,13.4,10000円,1ヶ月,無,ワンルーム,22.01m2,https://image1.homes.jp/smallimg/image.php?fil...,https://image1.homes.jp/smallimg/image.php?fil...,https://www.homes.co.jp/chintai/room/994725341...,港区


Google maps APIを使って物件の緯度経度を出す

In [25]:
import pandas as pd
import requests

API_KEY = "GoogleAPIキーを入力してください" 

def get_geocode_from_name(name):
    address = name.replace(" ", "+")  # 空白をプラスに置き換える
    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={API_KEY}"
    response = requests.get(url)
    data = response.json()
    if data["status"] == "OK":
        location = data["results"][0]["geometry"]["location"]
        lat = location["lat"]
        lng = location["lng"]
        return lat, lng
    else:
        return None, None


# 緯度経度の取得と格納
df_deduplicated['緯度'] = df_deduplicated['名称'].apply(lambda x: get_geocode_from_name(x)[0])
df_deduplicated['経度'] = df_deduplicated['名称'].apply(lambda x: get_geocode_from_name(x)[1])

In [26]:
df_deduplicated.head(5)

,名称,アドレス,アクセス,築年数,構造,階数,家賃,管理費,敷金,礼金,間取り,面積,物件画像URL,間取画像URL,物件詳細URL,区,緯度,経度
0,プラウドフラット虎ノ門,東京都港区西新橋３,"東京メトロ日比谷線 虎ノ門ヒルズ駅 徒歩7分, 都営三田線 御成門駅 徒歩7分",1年,15階建,2階,13.4,10000円,1ヶ月,無,ワンルーム,22.01m2,https://image1.homes.jp/smallimg/image.php?fil...,https://image1.homes.jp/smallimg/image.php?fil...,https://www.homes.co.jp/chintai/room/994725341...,港区,35.664507,139.751230
1,パークアクシス芝浦キャナル,東京都港区海岸３,"新交通ゆりかもめ 芝浦ふ頭駅 徒歩7分, 新交通ゆりかもめ 日の出駅 徒歩9分, JR山手線...",3年,10階建,10階,28.2,20000円,1ヶ月,無,2LDK,56.01m2,https://image.homes.jp/smallimg/image.php?file...,https://image4.homes.jp/smallimg/image.php?fil...,https://www.homes.co.jp/chintai/room/2d64f3e82...,港区,35.644665,139.755832
2,カナール芝浦,東京都港区芝浦２,"JR山手線 田町駅 徒歩9分, 都営浅草線 三田駅 徒歩12分, ゆりかもめ 芝浦ふ頭 徒歩10分",2年,8階建,3階,13.2,10000円,1ヶ月,1ヶ月,ワンルーム,30.88m2,https://image3.homes.jp/smallimg/image.php?fil...,https://image.homes.jp/smallimg/image.php?file...,https://www.homes.co.jp/chintai/room/4d6b446a5...,港区,35.645032,139.753236
4,京急本線 北品川駅 徒歩10分,東京都港区港南２,"京急本線 北品川駅 徒歩10分, 東京モノレール 天王洲アイル駅 徒歩12分, JR東海道本...",2年,15階建,8階,11.2,15000円,1ヶ月,無,ワンルーム,22.55m2,https://image4.homes.jp/smallimg/image.php?fil...,https://image4.homes.jp/smallimg/image.php?fil...,https://www.homes.co.jp/chintai/room/6be8adeeb...,港区,35.622041,139.739295
5,ＴＨＥ ＣＬＡＳＳ ＭＩＮＡＭＩＡＺＡＢＵ,東京都港区南麻布１,"東京メトロ南北線 麻布十番駅 徒歩8分, 都営三田線 白金高輪駅 徒歩12分",4年,11階建,10階,13.8,10000円,1ヶ月,1ヶ月,1K,26.1m2,https://image2.homes.jp/smallimg/image.php?fil...,https://image4.homes.jp/smallimg/image.php?fil...,https://www.homes.co.jp/chintai/room/15287862f...,港区,35.649991,139.730715


In [27]:
df_deduplicated.info()

<class 'pandas.core.frame.DataFrame'>
Index: 303 entries, 0 to 1365
Data columns (total 18 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   名称       303 non-null    object 
 1   アドレス     303 non-null    object 
 2   アクセス     303 non-null    object 
 3   築年数      303 non-null    object 
 4   構造       303 non-null    object 
 5   階数       303 non-null    object 
 6   家賃       303 non-null    float64
 7   管理費      303 non-null    object 
 8   敷金       303 non-null    object 
 9   礼金       303 non-null    object 
 10  間取り      303 non-null    object 
 11  面積       303 non-null    object 
 12  物件画像URL  303 non-null    object 
 13  間取画像URL  303 non-null    object 
 14  物件詳細URL  303 non-null    object 
 15  区        303 non-null    object 
 16  緯度       297 non-null    float64
 17  経度       297 non-null    float64
dtypes: float64(3), object(15)
memory usage: 45.0+ KB


In [28]:
import sqlite3

In [29]:
# データベース接続を作成
conn = sqlite3.connect('property.db')

In [30]:
# データフレームをSQLiteテーブルに変換
df_deduplicated.to_sql('SUUMOHOMES', conn, if_exists='replace', index=False)

303

In [31]:
conn.close()